In [ ]:
!pip install opencv-python-headless

import cv2
import numpy as np
import collections

# Constants for size, color, and distance filtering
MIN_RADIUS = 1
MAX_RADIUS = 10
BLACK_THRESHOLD = (0, 0, 0, 50, 50, 50)  # lower and upper bounds for black in BGR

def detect_circles(frame):
    """Detect circles using Hough Circle Transform."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=30,
        param1=60,
        param2=20,
        minRadius=MIN_RADIUS,
        maxRadius=MAX_RADIUS
    )
    return circles[0] if circles is not None else []

def filter_by_color_HSV(frame, circles):
    """HSV 색상 필터링으로 특정 색상의 공 탐지."""
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 흰색, 노란색, 핑크색 등 색상 범위 정의 (필요한 색만 사용 가능)
    color_ranges = {
        #'white': (np.array([0, 0, 200]), np.array([180, 30, 255])),
        'yellow': (np.array([20, 100, 100]), np.array([35, 255, 255])),
        #'pink': (np.array([140, 100, 100]), np.array([170, 255, 255])),
        # 'red': [
        #     (np.array([0, 100, 100]), np.array([10, 255, 255])),
        #     (np.array([170, 100, 100]), np.array([180, 255, 255]))
        # ],
        # 'blue': (np.array([100, 100, 100]), np.array([130, 255, 255])),
        'fluorescent_yellow': (np.array([25, 200, 200]), np.array([40, 255, 255])),
    }

    valid_circles = []
    for circle in circles:
        x, y, r = map(int, circle)

        # 원 내부 영역 가져오기 (ROI)
        roi = hsv_frame[max(0, y-r):min(y+r, hsv_frame.shape[0]), max(0, x-r):min(x+r, hsv_frame.shape[1])]
        if roi.size == 0:
            continue

        for color, bounds in color_ranges.items():
            if isinstance(bounds, list):
                # 두 개의 범위를 가진 색상 처리 (빨간색)
                mask1 = cv2.inRange(roi, bounds[0][0], bounds[0][1])
                mask2 = cv2.inRange(roi, bounds[1][0], bounds[1][1])
                mask = cv2.bitwise_or(mask1, mask2)
            else:
                # 일반 색상 범위 처리
                lower, upper = bounds
                mask = cv2.inRange(roi, lower, upper)

            # ROI에서 유효한 색상 비율 확인
            if cv2.countNonZero(mask) > 0.5 * (r**2 * np.pi):  # 원 면적의 50% 이상이 해당 색상일 경우
                valid_circles.append(circle)
                break  # 한 번 유효한 색상이 확인되면 다음 색상은 확인하지 않음

    return valid_circles



def measure_distance(prev_circle, current_circles):
    """Find the closest circle to the previous frame's circle."""
    if prev_circle is None:
        if current_circles is not None and len(current_circles) > 0:
          return current_circles[0]
        else:
          return None

    min_distance = float('inf')
    best_circle = None
    for circle in current_circles:
        dist = np.sqrt((circle[0] - prev_circle[0])**2 + (circle[1] - prev_circle[1])**2)
        if dist < min_distance:
            min_distance = dist
            best_circle = circle
    return best_circle


def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error opening video file {input_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    prev_circle = None
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        circles = detect_circles(frame)
        circles = filter_by_color_HSV(frame, circles)
        ball_circle = measure_distance(prev_circle, circles)

        if ball_circle is not None:
            x, y, r = map(int, ball_circle)
            cv2.circle(frame, (x, y), r, (0, 255, 0), 2)  # Draw the detected ball
            prev_circle = ball_circle

        out.write(frame)
        frame_number += 1

    cap.release()
    out.release()
    print(f"Processed video saved as {output_path}")

# Path to input video on Google Drive
input_video_path = '/content/drive/MyDrive/soccer/model_video.mp4'
# Path to save output video on Google Drive
output_video_path = '/content/drive/MyDrive/ball.mp4'

# Process the video
process_video(input_video_path, output_video_path)


Processed video saved as /content/drive/MyDrive/ball.mp4
